# Development Notebooks

## Create Points for StreamStats

Usage Notes:


Outputs:


by Shane Putnam: sputnam@dewberry.com & Seth Lawler: slawler@dewberry.com

In [1]:
import os
from osgeo import gdal, ogr,osr
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

### Load functions:

In [2]:
class StreamGrid(object):
    ''''''
    def __init__(self,path):
        self.path = path
        self.data = gdal.Open(path)
        self.band = self.data.GetRasterBand(1)
        self.array= self.band.ReadAsArray()

    def dataframe(self):
        '''Returns the flow accumulation grid as a pandas dataframe object'''
        df = pd.DataFrame(self.array)
        return df

    def transform(self,x,y):
        '''Method takes indices from dataframe or array and returns projected coordinates'''
        ul_x, x_dim, x_rotation, ul_y, y_rotation, y_dim = self.data.GetGeoTransform()
        x_coord = x * x_dim + ul_x + (x_dim / 2.)
        y_coord = y * y_dim + ul_y + (y_dim / 2.)
        coords = x_coord, y_coord
        return coords

def MoveUpstream(df, row, col, nogo_cells):
    '''Function searches surrounding cells in stream network grid to identify the
        grid cell immediately upstream. Output is index pair of upstream cell '''
    curr_cell_idxs = (row, col)
    
    current_cell= df[row][col]
    assert current_cell.values == 1
    surr_cells = []

    for i in range(-1,2):
        
        for j in range(-1,2):

            value = df[row + i][col+j] #Read Value of raster cell

            if value == 0:
                # if value is zero, no stream in this cell
                continue
                
            elif value == 1:
                # if value is 1, add to list
                surr_cells.append([row + i],[col+j])
                
            elif (row, col) in nogo_cells:
                continue
                
            else:
                print("Error?")
                
    return curr_cell_idxs, surr_cells

def UpstreamIterator(flowgrid, idx_n, idx_list,str_len):
    '''Algorithm created to iterate over steps to calculate streamline distance and find the
       uus point of interest (stopping point, defined by distance given in while expression in __main__) '''
    r,c = int(idx_n[0][0]),int(idx_n[1][0])
    idx_n = MoveUpstream(flowgrid.dataframe(),r,c)
    xypair = idx_n[0][0],idx_n[1][0]
    str_len += GetDistance(r,c, xypair[0], xypair[1], cellsize)
    usxy = flowgrid.transform(xypair[0],xypair[1])
    idx_list.append(usxy)
    return usxy, idx_n, str_len    

### Load the masked stream grid:

In [3]:
in_tif=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\rock_creek_clip.tif' #Load the stream grid raster which was maksed by the catchment polygon

In [63]:
sg = StreamGrid(in_tif)
proj = osr.SpatialReference(wkt=sg.data.GetProjection())
crs=proj.GetAttrValue('AUTHORITY',1)
print("epsg:",crs)

epsg: 5070


In [64]:
df = sg.dataframe()
df.replace(255,0, inplace=True)
df.head(n=2)

,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Specify the pour point to set the start location of the search:

In [65]:
def coord2index(sg, lat, lon):
    transform=sg.data.GetGeoTransform()
    pix_x = int((lon - transform[0]) / transform[1])
    pix_y = int((transform[3] - lat ) / -transform[5])
    return pix_x, pix_y

In [67]:
lat=1925315.186
lon=1616784.964

In [67]:
pix_x, pix_y =coord2index(sg, lat, lon)
pourpoint=[(pix_x, pix_y)]
print(pourpoint)

[(877, 1848)]


### Move up the stream and identify the confluences:

In [8]:
def MoveUpstream(df, stream_cell, nogo):
    row=stream_cell[0][0]
    col=stream_cell[0][1]      
    cell_value= df[row][col]
    assert cell_value == 1
                 
    nogo.append((row,col))
    stream_cell=[]

    for i in range(-1,2):
        
        for j in range(-1,2):

            value = df[row + i][col+j] #Read Value of raster cell
            
            if value == 0: # if value is zero, no stream in this cell
                continue
            
            elif value==1 and (row+i,col+j) not in nogo: # if value is 1, add to list
                stream_cell.append((row + i,col+j)) 
    return stream_cell, nogo

In [9]:
def FindConfluence(df, stream_cell, nogo): #Keep moving upstream until you find a confluence--problem: might hit a dead end, maybe add an else if it is the end?
    while len(stream_cell)==1:
        stream_cell, nogo=MoveUpstream(df, stream_cell, nogo)
    nogo=list(set(nogo+stream_cell))
    #nogo=nogo+stream_cell
    return stream_cell, nogo

In [10]:
def NextConfluence(df,confl,nogo,ppoints):
    stream_cell=[]
    confl1=[]
    stream_cell, nogo=MoveUpstream(df, [confl], nogo) #Move up one cell from the confluence
    stream_cell, nogo=MoveUpstream(df, stream_cell,nogo) #And move up one more time 
    ppoints=ppoints+stream_cell #Add the stream_cell that is located three up from the intial split point.
    confl1, nogo=FindConfluence(df,stream_cell,nogo)
    return confl1, nogo, ppoints

In [11]:
nogo=[]
ppoints=[]
stream_cell=pourpoint

In [12]:
confl, nogo=FindConfluence(df, stream_cell, nogo)

In [13]:
confl

[(873, 1802), (874, 1802)]

In [14]:
l=len(confl)-1
i=0
while i<=l:
    confl1=[]
    confl1, nogo, ppoints=NextConfluence(df,confl[i],nogo,ppoints)
    if len(confl1)==0:
        i+=1
    else:
        confl=confl+confl1
        l=len(confl)-1
        i+=1

IndexError: list index out of range

### Save the conflunces as a shapefile:

In [49]:
def index2coord(sg, confl):
    transform=sg.data.GetGeoTransform()
    longitude=[]
    latitude=[]

    for i in range(len(confl)):
        longitude.append(((transform[1]*confl[i][0])+transform[0])+transform[1]/2.)
        latitude.append((transform[3]-(confl[i][1]*-transform[5]))+transform[5]/2.)
    return longitude, latitude

def geodataframe(longitude, latitude, epsg):
    coord_df=pd.DataFrame(data={'Lon':longitude,'Lat':latitude})
    coord_df['Coordinates'] = list(zip(coord_df.Lon, coord_df.Lat))
    coord_df['Coordinates'] = coord_df['Coordinates'].apply(Point)
    gdf = gpd.GeoDataFrame(coord_df, crs={'init': 'epsg:%s' %epsg}, geometry='Coordinates')
    return gdf

In [50]:
longitude, latitude=index2coord(sg,confl)

In [51]:
gdf=geodataframe(longitude, latitude, crs)

gdf.to_file(filename = r'C:\Users\sputnam\Documents\GitHub\usgs-tools\results\allconfluences.shp')

gdf.head(2)

,Lon,Lat,Coordinates
0,1616745.0,1925775.0,POINT (1616745 1925775)
1,1616755.0,1925775.0,POINT (1616755 1925775)
